In [ ]:
#### Obter dados "Card Text:", "Flavor Text:", "Rarity:", "Artist:", "Card Number:
import os
import pandas as pd
import requests
import urllib3
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

HEADERS = {"User-Agent": "Mozilla/5.0"}

def obter_dados_cards(setUrl, Variation):
    session2 = requests.Session()
    session3 = None
    url = setUrl
    extracted_data = {}

    response = session2.get(url, headers=HEADERS, verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Encontrar variações do card
        card_variations_div = soup.find("div", class_="variations")
        if card_variations_div:
            variations = card_variations_div.find_all("a")
            if len(variations) > 1:
                variation_url = variations[Variation - 1]["href"]
                variation_url = variation_url.replace("/Pages", "https://gatherer.wizards.com/Pages")
                url = variation_url

                session3 = requests.Session()
                response = session3.get(url, headers=HEADERS, verify=False)

        session2.close()

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table_div = soup.find("div", class_="smallGreyMono", style="margin-top: 5px;")
        
        if table_div:
            rows = table_div.find_all("div", class_="label")
            values = table_div.find_all("div", class_="value")
            extracted_data = {row.text.strip(): value.text.strip() for row, value in zip(rows, values)}
    else:
        print(f"Erro ao acessar a página: {url}")

    if session3 is not None:
        session3.close()

    return extracted_data

# Nome do arquivo Excel
excel_file = r"C:\Users\macra\Documents\00Python\Web Scrapping\cards_data pt1.xlsx"

# Carregar o Excel
df = pd.read_excel(excel_file)

# Colunas que queremos preencher
columns_to_fill = ["Card Text:", "Flavor Text:", "Rarity:", "Artist:", "Card Number:"]

counter = 0

# Percorrer os registros
for index, row in df.iterrows():
    counter += 1
    url = row["Link"]
    Variation = row["Variações"]
    rarity = row["Rarity:"]

    # Verificar se "rarity" está ausente ou vazio
    if pd.isna(rarity) or str(rarity).strip() == "":
        try:
            extracted_data = obter_dados_cards(url, Variation)

            # Atualizar os valores no DataFrame se estiverem vazios
            for col in columns_to_fill:
                if pd.isna(row[col]) or row[col] == "":
                    df.at[index, col] = extracted_data.get(col, "")

        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar {url}: {e}")
        except Exception as e:
            print(f"Erro inesperado na linha {index}: {e}")

    # Salvar periodicamente para evitar perda de dados
    if counter % 100 == 0:
        df.to_excel(excel_file, index=False)
        print(f"Salvando progresso... {counter} registros processados.")
    
    if counter % 400 == 0:
        quit()
        os.system("Dados_Cards_MTG.ipynb")

# Salvar as alterações de volta no Excel
df.to_excel(excel_file, index=False)
print("Atualização concluída!")

In [ ]:
#### Preencher card number ####
import os
import pandas as pd
import requests
import urllib3
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

HEADERS = {"User-Agent": "Mozilla/5.0"}

def obter_dados_cards(setUrl, Variation):
    session2 = requests.Session()
    session3 = None
    url = setUrl
    extracted_data = {}

    response = session2.get(url, headers=HEADERS, verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Encontrar variações do card
        card_variations_div = soup.find("div", class_="variations")
        if card_variations_div:
            variations = card_variations_div.find_all("a")
            if len(variations) > 1:
                variation_url = variations[Variation - 1]["href"]
                variation_url = variation_url.replace("/Pages", "https://gatherer.wizards.com/Pages")
                url = variation_url

                session3 = requests.Session()
                response = session3.get(url, headers=HEADERS, verify=False)

        session2.close()

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table_div = soup.find("div", class_="smallGreyMono", style="margin-top: 5px;")
        table_div_ = soup.find_all("div", class_="smallGreyMono", style=None)

        if table_div:
            # Encontrando todas as labels e valores dentro da div
            rows = table_div.find_all("div", class_="label")
            values = table_div.find_all("div", class_="value")

            # Verificando se existe a label "Card Number:" e obtendo seu valor correspondente
            card_number_value = None
            for row, value in zip(rows, values):
                if row.text.strip() == "Card Number:":
                    card_number_value = str(value.text.strip())
                    break  # Para o loop ao encontrar a primeira ocorrência
        elif table_div_:
            # Encontrando todas as labels e valores dentro da div
            rows = table_div_[0].find_all("div", class_="label")
            values = table_div_[0].find_all("div", class_="value")

            # Verificando se existe a label "Card Number:" e obtendo seu valor correspondente
            card_number_value = None
            for row, value in zip(rows, values):
                if row.text.strip() == "Card Number:":
                    card_number_value = str(value.text.strip())
                    break  # Para o loop ao encontrar a primeira ocorrência

        # if table_div:
        #     rows = table_div.find_all("div", class_="label")
        #     values = table_div.find_all("div", class_="value")
        #     extracted_data = {row.text.strip(): value.text.strip() for row, value in zip(rows, values)}
    else:
        print(f"Erro ao acessar a página: {url}")

    if session3 is not None:
        session3.close()

    # return extracted_data
    return card_number_value

# Nome do arquivo Excel
excel_file = r"C:\Users\macra\Documents\00Python\Web Scrapping\cards_data pt1.xlsx"

# Carregar o Excel
df = pd.read_excel(excel_file)

columns_to_fill = ["Card Number:"]
counter = 0

# Percorrer os registros
for index, row in df.iterrows():
    counter += 1
    url = row["Link"]
    Variation = row["Variações"]
    CardNumber = row["Card Number:"]

    # Verificar se "rarity" está ausente ou vazio
    if pd.isna(CardNumber) or str(CardNumber).strip() == "":
        try:
            extracted_data = obter_dados_cards(url, Variation) 
            if pd.isna(extracted_data):
                extracted_data = "-"
            else:
                extracted_data

            for col in columns_to_fill:
                if pd.isna(row[col]) or row[col] == "":
                    df.at[index, col] = extracted_data
                    # df.at[index, col] = extracted_data.get(col, "")

        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar {url}: {e}")
        except Exception as e:
            print(f"Erro inesperado na linha {index}: {e}")

    # Salvar periodicamente para evitar perda de dados
    if counter % 43 == 0:
        df.to_excel(excel_file, index=False)
        print(f"Salvando progresso... {counter} registros processados.")
        break

    if counter % 400 == 0:
        quit()
        os.system("Dados_Cards_MTG.ipynb")
        
# Salvar as alterações de volta no Excel
df.to_excel(excel_file, index=False)
print("Atualização concluída!")